In [ ]:
!pip install -q torch==1.6.0

# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch
from torch import nn

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# Работа с изображениями и сверточные сети

## Загружаем данные с Kaggle

Если вам еще не довелось работать с Kaggle, то сейчас мы начнем работу. Если вы работаете на своей машине (подняли свой Jupyter сервер), то процесс может варьироваться. То есть вы можете сами скачать данные с Kaggle и положить в нужную папку. Сейчас мы рассмотрим процесс работы с Kaggle на платформе Google Colab!

Соревнование, по которому мы будем работать, называется [Digit Recognizer](https://www.kaggle.com/c/digit-recognizer/data).

Первое, что нужно сделать - установить пакет kaggle с PyPI, чтобы можно было пользоваться их утилитами:

In [ ]:
!pip install -q kaggle==1.5.6

После того, как пакет установлен, нам нужно зайти в свой аккаунт на Kaggle (в настройки аккаунта) и получить файл `kaggle.json` с сайта. Его можно получить на машину переходом в настройки аккаунта и там кликом на кнопку "Create New API Token":

![KaggleAPI](https://docs.google.com/uc?export=download&id=1BAin4N_I5XZ-hmQMDtkfRohlT5IIWROS)

Это выгрузит на ваш компьютер нужный файл. Затем нужно загрузить этот файл в сессию! 
> Специфика здесь в том, что все вычисления Google Colab производит у себя на серверах, поэтому, если хочется поработать в файлами, то их надо загрузить на этот сервер.

Сделать это можно нажатием кнопки "Загрузить в сессионное хранилище" в меню слева:

![ColabUpload](https://docs.google.com/uc?export=download&id=1bG5S7HINusAwmy6tdiOX15zB3h9ah6Lc)

Или программно:
```
from google.colab import files
files.upload()
```

Этот код вызовет окошечко загрузки, где можно указать, какой файл загрузить.

> Обратите внимание, что перезапуск среды может стереть все загруженные файлы, поэтому нужно загружать этот файл каждый раз, когда хотите загрузить данные с Kaggle в сессию Google Colab.

После этого нужно разместить файл по пути `~/.kaggle/kaggle.json` и можно пользоваться утилитами, а именно скачать набор данных, которым мы воспользуемся - MNIST:

In [ ]:
# Указываем, где лежит kaggle.json и скачиваем датасет
!export KAGGLE_CONFIG_DIR=$PWD && kaggle competitions download -c digit-recognizer

В результате мы можем увидеть архив в нашей папке `digit-recognizer.zip`. Самое время его распаковать и посмотреть, что там:

In [ ]:
# Создадим папку для набора данных
!mkdir -p dataset && unzip -qn digit-recognizer.zip -d dataset
!ls dataset

Отлично, мы видим данные в виде файлов `train.csv` и `test.csv`. Файл `sample_submission.csv` нужен как демонстрация формата для загрузки решения для соревнований. Вы можете попытать свои силы в различных соревнованиях! В данной практике мы этот аспект опустим и попытаемся посмотреть, что из себя представляют данные: 

In [ ]:
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

In [ ]:
train_df.head()

In [ ]:
print(train_df.shape)
print(train_df.columns)
print(sorted(train_df['label'].unique()))

In [ ]:
train_df['label'].hist(bins=10)

In [ ]:
test_df.head()

In [ ]:
print(test_df.shape)
print(test_df.columns)

Беглым просмотром мы видим, что данные для обучения имеют разметку цифр от 0 по 9, всего в тестовом наборе 42000 записей. Распределение классов близко к равномерному - отлично. Тестовый набор не имеет разметки (вероятнее всего нужен, чтобы делать предсказания и заливать на сайт для соревнований), поэтому мы не сможем его использовать для оценки метрик. Тем не менее, мы можем воспользоваться им для визуальной оценки работы модели.

Описание на сайте дает нам информацию о том, что данные для работы - изображения, но представлены в виде колонок `pixel0` - `pixel783`, то есть всего 784 пикселя. Что-то новенькое для нас! Давайте разберемся, что такое изображения и как их понимать!

# Вводная по работе с изображениями

Итак, мы подошли к вопросу, а как же работать с изображениями? Для ответа на этот вопрос нам нужно понять, как изображения представляются в цифровом виде и после уже мы поймем, как их обрабатывать глубокими нейросетями!

В компьютере изображения представлены в виде матриц. Как и привычно, изображения имеют ширину (в пикселях) и высоту (в пикселях). Но матрица, которой представляют цветное изображения чаще всего имеет три размерности! Давайте взглянем на картинку:

<center><img src="https://docs.google.com/uc?export=download&id=1imEGBNfuOsnqGJvte-LilcFkNohxH4-V"/></center>

Она многое содержит, но главная суть в том, что цветные изображения - это три двумерные матрицы, каждая из которых является отдельным каналом (красный, зеленый или синий). Черно-белое изображение - это одна двумерная матрица.

С ЧБ все достаточно просто, каждый пиксель - это значение в диапазоне $[0; 255]$ (8-бит), которое отражает яркость в этом пикселе.

С цветным все чуть сложнее, каждый пиксель содержит три значения. Первое - количество красного, второе - зеленого, третье - синего. Смешав цвета с разными интенсивностями, можно получить различные цвета (а именно 16777215 вариантов цветов - 24 бита). То есть по сути каждый пиксель можно представить вот так:

<center><img src="https://docs.google.com/uc?export=download&id=1YluceotxuoE0w5Wn3FWe7hFdmYHjCxuZ" /></center>

На этом и основывается принцип представления изображений! Это всего лишь двумерная матрица, если изображение черно-белое и трехмерная матрица, у которой последняя размерность равна трем.

> Например, цветное изображение размером 1024х768 - это матрица с размерностью $(768, 1024, 3)$. Ведь сначала идут ряды матрицы (высота), затем колонки (ширина), а затем количество каналов. Такая схема называется HWC ~ Height, Width, Channel.



# Смотрим данные

Из описания данных мы знаем, что картинки цифр в датасете представлены в виде 28х28 серых изображений, а это как раз всего 784 пикселя! Значит, в теории, мы можем взять одну запись из данных и посмотреть на изображение? Проверим!

In [ ]:
# Первая (нулевая) запись в данных
#   Так как колонка label первая, то мы ее так можем пропустить
sample_row = train_df.iloc[0, 1:]
sample_label = train_df.iloc[0, 0]
# flattened - формат, когда многомерные данные представлены 
#   в виде одномерного массива
digit_flattened = sample_row.to_numpy()
digit_img = digit_flattened.reshape(28, 28).astype(np.uint8)

# imshow() - функция для отображения изображений
plt.imshow(digit_img, cmap='gray')
plt.title(f'Цифра: {sample_label}')
plt.xticks([])
plt.yticks([])
plt.show()

Отлично! Мы действительно видим цифру один на этом рисунке, как и заверяет нас разметка (истинное значение)!

Давайте теперь весь набор данных приведем к формату NHWC, в котором:
- N - количество записей в батче;
- H - высота изображений в батче;
- W - ширина изображений в батче;
- C - количество каналов в изображениях.

In [ ]:
imgs_batch_flattened = train_df.iloc[:, 1:].to_numpy()
imgs_batch = imgs_batch_flattened.reshape(-1, 28, 28, 1).astype(np.uint8)
imgs_batch_labels = train_df.iloc[:, 0].to_numpy()

print(imgs_batch.shape)
print(imgs_batch_labels[:5])

С таким видом уже проще работать, так как не нужно преобразовывать из flattened формата в изображение.

## Задание - более широкий взгляд

Отобразите 25 случайных цифр из набора данных в матрице 5х5 с подписями.

In [ ]:
# TODO - отобразите цифры в виде матрицы рисунков 5х5
# NOTE - для отображения plt.imshow() серого изображения 
#   нужно убрать одну размерность, так как он принимает (28, 28),
#   а на (28, 28, 1) выдает ошибку
# Сделать это можно путем img[:, :, 0] или img[..., 0], что равнозначно

# Подготовка изображений

Как уже ранее обсуждалось, PyTorch работает с собственным форматом - тензоры, а это значит, что нам надо подготовить изображения из нынешнего представления в формат для PyTorch.

Для этого необходимо сделать несколько действий с готовым изображением:
- Преобразовать изображение к формату тензора типа `float`;
- Поделить изображение на 255, чтобы привести к диапазону $[0; 1]$;
> Данное требование обосновано необходимостью нормирования данных при работе с нейросетями - при нормировании данных сеть учится быстрее и меньше попадает на плато в данных.
- Изменить формат от стандартного HWC к CHW - переставить порядок в матрице.
> Это требование работы со сверточными сетям в PyTorch

Напишем полный код предобработки изображения:

In [ ]:
img = imgs_batch[1]

# Преобразуем к тензору и затем к типу float
img_tnsr = torch.tensor(img).float()
# Делим на 255, чтобы привести к единичному диапазону
img_tnsr = img_tnsr/255.
# Метод .permute() - аналог np.transpose(), 
#   который позволяет поменять порядок размерностей в массиве
# В этом методе указываются индексы размерностей в том порядке, 
#   как их надо поставить
# Изначально было H~0, W~1, C~2
# Мы хотим поставить C, H, W ~ 2, 0, 1
img_tnsr = img_tnsr.permute(2, 0, 1)

print(img_tnsr.shape)

Отлично! Данные приведены к требуемому формату! Теперь настало время научиться работать с классом датасета в PyTorch для загрузки данных.

# Датасет в формате PyTorch

Мы уже сталкивались с разными вспомогательными инструментами PyTorch, а сейчас мы познакомимся с инструментом подгрузки данных. Суть его заключается в том, что мы можем написать реализацию нашего собственного класса с необходимым функционалом подготовки всего одной записи из данных к нужному виду. Далее другие инструменты, используя написанную реализацию, будут загружать целые пачки данных необходимого количества.

От слов к делу! Начнем с простой реализации, чтобы понять смысл класса:

In [ ]:
class MySimpleDataset(torch.utils.data.Dataset):
    # Аргумент count здесь для примера, 
    #   можно делать любые аргументы конструктору
    def __init__(self, count):
        self.numbers = np.arange(count)
    
    # Метод __len__() позволяет применять функцию len() в объекту
    #   такого класса
    def __len__(self):
        return len(self.numbers)

    # Метод __getitem__() позволяет обращаться к объекту
    #   такого класса по индексу
    def __getitem__(self, index):
        return torch.tensor([self.numbers[index]])

Вот так выглядит минималистичный пример класса датасета. В нем реализованы все необходимые методы. По сути необходима реализация метода `__len__()` для получения размера датасета и метода `__getitem()__` для получения данных из датасета по индексу. Посмотрим, как обращаться с этим датасетом:

In [ ]:
my_dataset = MySimpleDataset(10)

# Вот здесь будет вызван метод __len__() класса
# Так можно получить размер датасета
print(len(my_dataset))

# Вот здесь будет вызван метод __getitem__() класса
# Таким образом можно получить данные из датасета
print(my_dataset[3])

Теперь давайте посмотрим на инструмент `torch.utils.data.DataLoader` (https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), который позволяет на основе датасета реализовывать получение батчей данных, перемешивание и много чего другого.

In [ ]:
# Создадим объект загрузчика
my_dataloader = torch.utils.data.DataLoader(
    dataset=my_dataset, # Передаем наш датасет в загрузчик 
    batch_size=4,       # Задаем размер батча для подгрузки
    shuffle=True        # Сообщаем о том, что нам нужны перемешанные данные
)

# Размер загрузчика представляется уже в количестве батчей
#   4+4+2 (остаток)
print(len(my_dataloader))

# Посмотрим, как загрузчик выдает данные
# Так как он не имеет индексации, то один из простых способов - 
#   пройти по загрузчику циклом
for batch in my_dataloader:
    print('----')
    print(batch)
    print(batch.shape)

Таким образом, мы написали класс для подготовки тензора по одной записи в данных, а загрузчик уже объединяет их в батчи. Причем, наш датасет выдает массив размером $(1)$, а загрузчик объединяет такие данные и добавляет размерность $(4, 1)$. То есть из формата $K$ делается формат $NK$, где $K$ - размер данных, выдаваемых датасетом, $N$ - размер батча или остатка от него.

Это очень удобно, так как мы пишем реализацию подготовки наших картинок, а далее инструменты объединяют их в батчи и при необходимости перемешивают между собой.

Давайте теперь вернемся к нашей задаче распознавания цифр и напишем свой датасет с блекджеком и подготовкой изображений!

## Задание - пишем датасет изображений

В конструктор датасета будет передан DataFrame с тренировочными изображениями. Задача - написать реализацию датасета, чтобы он могу выдавать изображения в подготовленном формате! 

In [ ]:
# TODO
class DigitsDataset(torch.utils.data.Dataset):
    def __init__(self, digits_df):
        # Прямо в конструкторе преобразуйте DataFrame в
        #   массив изображений и вектор разметки

    def __len__(self):
        # Верните количество изображений в наборе
        return self.imgs.shape[0]

    def __getitem__(self, index):
        # Получите изображение и разметку по индексу
        
        # Подготовьте изображение для работы в PyTorch
        #   float, [0; 1], CHW

        # Верните tuple, состоящий из двух элементов:
        #   - тензор изображения
        #   - значение разметки (истинное значений цифры на изображении)
        return img_tnsr, label

In [ ]:
# TEST
digits_dataset = DigitsDataset(train_df)

img_tnsr, label = digits_dataset[100]

assert isinstance(img_tnsr, (torch.Tensor))
assert np.all(img_tnsr.shape == torch.Size((1, 28, 28)))
assert np.isclose(img_tnsr.max(), 1)
assert np.isclose(img_tnsr.min(), 0)
assert label == 9

img = img_tnsr.permute(1, 2, 0).numpy()
img = (img*255).astype(np.uint8)

plt.imshow(img[..., 0], cmap='gray')
plt.title(f'Цифра: {label}')
plt.xticks([])
plt.yticks([])
plt.show()

Если в результате теста вы видите цифру девять - все подготовлено верно! Теперь можно создать загрузчик и убедиться, что данные выдаются в виде батчей:

In [ ]:
digits_dataloader = torch.utils.data.DataLoader(
    dataset=digits_dataset,
    batch_size=4,
    shuffle=False
)

for imgs_batch, labels_batch in digits_dataloader:
    print(imgs_batch.shape)
    print(labels_batch)

    # Просто проверка, не нужно проходить по всему!
    break

# Еще для проверки можно преобразовать загрузчик в итератор 
#   и вызвать функцию next()
print('---')
imgs_batch, labels_batch = next(iter(digits_dataloader))
print(imgs_batch.shape)
print(labels_batch)

Замечательно! На этом подготовка изображений завершена! Мы бегло выполнили анализ данных, преобразовали в нужный формат и написали класс датасета, который позволяет получать данные в формате, требуемом для обучения! Самое время перейти к разработке модели сверточной сети!

# Из чего состоит сверточная нейросеть

Модели сверточных сетей считаются глубокими сетями и относятся к области глубокого обучения. Почему? Причина именно в том, что работа ведется с изображениями.
> Тут уточнение, что сверточные нейросети уже давненько и к обработке звука применили, но это за рамками темы.

Основная идея заключается в том, что ранее мы работали с данными, признаки которых представляли собой конкретные показатели и по ним принималось то или иное решение моделью. В случае изображений признаками являются интенсивности пикселей, но по отдельности они несут крайне бесполезную информацию. Совокупностью пикселей хотя бы в небольшой области может дать уже намного бОльшую информацию, именно поэтому сверточные сети показали великолепные результаты, так как сама операция свертки (из названия) производит обработку изображения по областям.

Глубоким обучением данная область считается, потому что сеть сама ищет и генерирует признаки, исходя из данных. В этом плане с одной стороны проще, так как не требуется большой работы с предобработкой признаков, с другой - многое теперь зависит от модели, количества данных (а для глубокого обучения их нужно больше) и подготовки этих данных путем анализа шума и фильрации выбросов.

Схематично сверточную сеть можно представить следующим образом:

![image_rgb](https://docs.google.com/uc?export=download&id=1pHmlOy4MC61BnZGRAFFTmt6_VUJvMPsV)

Видно, что она состоит из двух частей:
- Feature learning - генерация признаков, на этом этапе модель состоит только из сверточных слоев, на которых генерируются признаки изображения;
- Classification - классификация как частный случай, скорее часть принятия решения, ставится обычная полносвязная нейросеть, которая на основе сгенерированных из изображения признаков решает конечную задачу.

Видно, что для примера представлена задача классификации, на вход (слева) поступает изображение, а задача модели - классифицировать эту картинку. Вообще, мы такую задачу и планируем сейчас решать, только наши предсказываемые классы - цифры от 0 до 9, то есть 10 классов!

Давайте теперь пройдемся по основным составляющим модели сверточной сети, причем мы опустим последний слой, так как ничего нового мы о нем не узнаем.

## Операция свертки

> Спасибо [статье](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) за картинки!

Операция свертки известна из многих областей, но в обработке изображения она выглядит следующим образом:

![Текст ссылки](https://miro.medium.com/max/535/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

> Если у вас не отображается гифка - скопируйте ссылку в браузер

Так что здесь проиходит? На изображении есть ядро размером $(3, 3)$, которое проходит по всему изображению слева направо на самом верху, затем опускается на один пиксель вниз и далее слева направо. Ядро имеет свои значения, пиксели на изображении тоже. Операция свертки заключается в том, что ядро, попадая на область изображения вычисляет сумму произведений поэлементных значений ядра и элементов области. Далее эта сумма записывается в результат. Давайте разберем верхний правый угол. Ядро имеет вид:
$$
\begin{bmatrix}
0 && 1 && 2 \\
2 && 2 && 0 \\
0 && 1 && 2
\end{bmatrix}
$$

Верхняя правая область имеет вид:
$$
\begin{bmatrix}
2 && 1 && 0 \\
1 && 3 && 1 \\
2 && 2 && 3
\end{bmatrix}
$$

Таким образом, сумма элементов будет равна:
$$
\sum = 0*2 + 1*1 + 2*0 + \\
2*1 + 2*3 + 0*1 + \\
0*2 + 1*2 + 2*3 = 1 + 8 + 8 = 17
$$

Результат 17 так и записан в верхний правый элемент результата! Отлично, все сошлось!

Теперь, в чем тут идея? Да именно в том, что сеть анализирует ядро области изображения и "преобразует" информацию, тем самым создавая новые "внутренние" представления для генерации признаков.

Но как слой свертки учится? В нейроне были веса входа, а здесь непонятно. Легко запомнить - изображение меняться не может, так что это фиксированная величина, поэтому ядро слоя свертки является обучаемым. Точнее, значения в ядре являются параметрами слоя и в ходе обучения настраиваются.

> Размер ядра - гиперпараметр операции свертки (слоя сети).


Теперь надо затронуть еще пару аспектов свертки, так как эта операция является основополагающей в сверточных сетях!

### Паддинг (отступы)

Самые наблюдательные обратят внимание, что все писели изображения кроме крайних могут попасть в центр ядра и активно участвуют в операции свертки. Крайние пиксели участвую в операции свертки реже, чем остальные пиксели, что сказывается на получении информации с краев изображения - плохо, ведь крайние пиксели могут нести полезную информацию. А что, если размер ядра $(11, 11)$? Тогда не один ряд крайних не участвует, а целых 5 рядов!

> Вам понятно, как 5 было получено? Ядро специально берется нечетного размера, чтобы был "центральный пиксель". Таким образом в стандартном виде 5 пикселей с краю никогда не станут центральными (размер ядра/2 с округлением вниз).

Так вот паддинг, как он выглядит:

![Текст ссылки](https://miro.medium.com/max/395/1*1okwhewf5KCtIPaFib4XaA.gif)

Идея заключается в добавлении отступов с краев, чтобы все пиксели в изображении равномерно участвовали в формировании полезной информации! Размер отступов определяется размером ядра в слое свертки. По сути, паддинг несет два назначения:
- Позволяет получать информацию со всего изображения без исключения краев;
- Позволяет формировать результат такой же по размеру, как и вход.

> Сверточная сеть составлена из операций свертки, поэтому входом не всегда являются изображения, а чаще всего входом является выход предыдущего слоя свертки.

> Паддинг чаще всего выбирается исходя из размера ядра, но может также считаться гиперпараметром слоя.

### Stride (шаг свертки)

Вторая особенность для обсуждения - шаг, с которым свертка проходит по изображению. В оригинале мы рассмотрели шаг равный единице, когда свертка проходит пиксель за пикселем. Можно в качестве страйда выбрать шаг равный двум пикселям, тогда это будет выглядеть следующим образом:

![Текст ссылки](https://miro.medium.com/max/294/1*BMngs93_rm2_BpJFH2mS0Q.gif)

Как видно, между шагами появился пропуск, а также результат стал заметно меньше! Это как раз характеризует страйд. При сравнении шага равного единице и двум, можно отметить, что вычисление производится быстрее, а также результат содержит меньше информации. В некоторых случаях это хорошо, тем более мы скоро рассмотрим слой отвечающий за прореживание информации, но выбор шага определяет размер выхода (во сколько раз он будет уменьшен). Например, размер изображения 400х400, страйд выбирается равный двум, так что результат будет иметь размер 200x200.

> Страйд является гиперпараметром слоя свертки.



### Dilation (разреженность)

Этот аспект свертки является достаточно специфичным и связан с таким понятием как **поле восприятия (receptive field)**. Данное понятие в данный момент рассматривать не будет, лишь отметим суть показателя разреженности:

![Текст ссылки](https://miro.medium.com/max/395/0*3cTXIemm0k3Sbask.gif)

Стандртная свертка имеет dilation rate равный единице. Анимация показывает суть работы при показателе равном двум. То есть ядро растягивается сильнее и получает пропуски в себе же. Информация таким образом собирается шире (это если кратко про поле восприятия и про то, что так оно расширяется), так как часто соседние пиксели не несут столько полезной информации, сколько несет более широкая зона, но с пропусками пикселей через один.

> Dilation rate также является гиперпараметром


## Свертка в PyTorch

В качестве небольшой практики попробуем операцию свертки отдельно ото всех остальных, чтобы немного разобраться в ней. Как всегда создаем модуль, отвечающий за эту операцию:



In [ ]:
conv_mod = nn.Conv2d(
    in_channels=1,  # Количество входных каналов = 
                    #   количество выходных каналов предыдущей свертки 
                    #   (или каналов входного изображения)
    out_channels=1, # Количество выходных каналов = количество ядер
    kernel_size=3,  # Размер ядра
    stride=1,       # Страйд по-умолчанию,
    padding=0       # Нулевой отступ
)

Тут важно отметить, что количество выходных канало определяет количество различных ядер внутри свертки. Так, например, даже если входной канал один, то выходных может быть 16, тогда внутри свертки будут обучаться 16 разных ядер, которые применяются каждое ко входу и формируют 16 разных выходов.

Теперь давайте проверим атрибуты модуля и попробуем применить свертку к случайному изображению:

In [ ]:
print(conv_mod.weight)

In [ ]:
X_rnd_img = torch.randint(low=0, high=2, size=(1, 1, 5, 5)).float()
print(X_rnd_img)

In [ ]:
# Заполним веса единицами и выполним операцию
conv_mod.weight.data.fill_(1)
# В свертке есть bias, его мы обнулим
conv_mod.bias.data.fill_(0)
print(conv_mod.weight)
print(conv_mod.bias)

result = conv_mod(X_rnd_img)
print(result)

По результатам можно увидеть вычисление результата, соответствующее ожиданиям расчета. Мы не обсуждали bias, но тут все просто - это просто число для каждого ядра, которое прибавляется ко всему ядру.

Таким образом, по аналогии с полносвязной сетью, где мы создавали линейный слой нейронов, здесь мы создаем линейный слой ядер. К чему эта аналогия? Для свертки тоже нужна нелинейность, рассмотрим одну из известных для сверточных сетей нелинейность - ReLU!

## Нелинейность для свертки

Как мы уже изучали, нейронная сеть без нелинейности - GPU на ветер. Поэтому глянем на штуку под названием Rectified Linear Unit (ReLU). Эта нелинейность выглядит очень просто, тем не менее является полноценной нелинейностью и показывает отличные результаты, хотя и на сегодняшний день есть огромное количество аналогов!

Выглядит она следующим образом:

In [ ]:
x = np.linspace(-10, 10, 300)
y = np.maximum(x, 0)

plt.plot(x, y)
plt.title('ReLU')
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Даже не будем писать формулу, просто максимум между значением и нулем. Наверное, было бы правильно рассмотреть ее производную, всякие свойства, но мы просто попробуем применить ее в PyTorch:

In [ ]:
x_tnsr = torch.tensor([-2, -1, 0, 1, 2]).float()
relu_op = nn.ReLU()

relu_op(x_tnsr)

Вот и все, нелинейность очень простая, тем не менее ее важно размещать в сверточной нейронной сети, чтобы добавить ей свойство нелинейности.

## MaxPooling

Пулинг в сверточной нейронной сети является этапом уменьшения размерности вместе со сжатием информации. Представить его можно следующим образом:

![Текст ссылки](https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png)

По сути, это ядро, которое не имеет значений, а просто берет максимум из области, на которое попадает и шагает со страйдом, равным размеру ядра. На изображении показан MaxPooling с ядром размером $(2, 2)$.

Данная операция является очень полезной с точки зрения преобразования данных из изображения в вектор признаков, на основе которого полносвязная сеть принимает решения. В сверточных сетях происходит постепенный рост размерности по каналам (C) и постепенное уменьшение размерности по сторонам (HW). Таким образом, картинка трансформируется в одномерный вектор, при этом этот вектор уже представляет собой не пространственную информацию, а информацию в виде вектора признаков, удобном для принятия решения по конкретной задаче.

> Размер ядра (размер пулинга) является гиперпараметром слоя

Думаю, стоит сразу перейти к практике с модулем в PyTorch:

In [ ]:
pool_mod = nn.MaxPool2d(
    kernel_size=2,  # Размер ядра пулинга
    # stride=2      # Можно задать страйд, но он по-умолчанию равен размеру ядра
)

x_rnd_img = X_rnd_img = torch.randint(low=0, high=20, size=(1, 1, 4, 4)).float()
print(x_rnd_img)

In [ ]:
result = pool_mod(x_rnd_img)
print(result)

Вот так мы быстренько рассмотрели три основных составляющий сверточной нейронной сети. По сути из них только операция свертки является обучающей, остальные две добавляют сети свойств, благодаря которым она может уверенно обучаться и давать предсказания с высокой точностью.

> Рассмотренные компоненты сверточной сети являются минимумом из существующих сегодня. Такие вещи как регуляризация с помощью BatchNorm, восстановление изображения с помощью Deconvolution и другие, опущены в данной практике, но тем не менее рекомендованы к изучению в дальнейшем! 

# Составим нашу первую сверточную сеть!

Сеть мы составим простую, будет всего два слоя, каждый из которых будет иметь нелинейность и слой пулинга. 

После этих слоев нам нужно преобразовать тензор с тремя размерностями CHW в тензор с одной размерностью за счет flattening.

In [ ]:
class MyFirstConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Делаем первый блок свертки
        # Воспользуемся классом nn.Sequential, чтобы сделать
        #   один модуль из нескольких
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # После пулинга размер из 28х28 преобразуется в 14х14

        # Делаем второй блок свертки
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # После пулинга размер из 14х14 преобразуется в 7х7
        
        # fc ~ fully connected block
        self.fc = nn.Sequential(
            # Первый слой для входа - это выпрямленный вектор
            # Размер выбирается исходя из предыдущего слоя
            # H*W*C, HxW = 7x7, C = кол-во каналов на пред. слое
            nn.Linear(7*7*32, 500),
            nn.Sigmoid(),
            # Выход последнего слоя по количеству предсказываемых классов
            # Размерность выхода - NK (K=10)
            nn.Linear(500, 10)
            # На этом уровне предсказаны Logits - сырые степени уверенности
            # Далее мы разберем, что функцию потерь это и ожидает, поэтому
            #   на этом блок закончен
        )
    
    def forward(self, x):
        # Делаем предсказания блоков
        out = self.layer1(x)
        out = self.layer2(out)
        # Делаем flattening результатов
        # Из NCHW делаем NV
        # N - размер батча, V = C*H*W
        out = out.reshape(out.size(0), -1)
        # Предсказания последним блоком
        out = self.fc(out)
        return out

Теперь, после написания класса важно сделать проверку того, что связи настроены верно, а если нет, то можно будет быстро отладить, пока не написан большой код.

In [ ]:
# TEST
model = MyFirstConvNet()

# Сделаем случайный батч для проверки
# 2 записи в батче, CHW - соответсвует нашим картинкам
X_rndm_batch = torch.randn((2, 1, 28, 28), dtype=torch.float)
y_pred = model(X_rndm_batch)

assert np.all(y_pred.shape == torch.Size((2, 10)))

# Функция потерь

Отлично! Наша модель написана и протестирована! Самое время разобраться с тем, как нам оценивать работу модели - копаем в [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [ ]:
# Создаем модуль
loss_op = nn.CrossEntropyLoss()

# Для начала сделаем идеальный случай
# Тензор истинных значений должен быть одномерным c размером N и в нем указываются
#   индексы классов
y_true = torch.tensor([1])
# Тензор предсказаний должен быть NK (K - кол-во классов) 
#   с сырыми степенями уверенности
# Для примера возьмем K=4, N=1

# Вариант 1, степень уверенности у индекса 3 выше всех, плохое предсказание
y_pred_1 = torch.tensor([[0, 0, 0, 2]]).float()
# Вариант 2, индекс 0 выше всех, но также имеются уверенности у 2 и 3,
#   хуже, чем вариант 1, так как степень уверенности выше у 0,
#   чем в первом варианте у 3
y_pred_2 = torch.tensor([[10, 0, 3, 1]]).float()
# Вариант 3, наибольший индекс 1, также имеются уверенности у остальных,
#   то что надо
y_pred_3 = torch.tensor([[10, 100, 3, 1]]).float()

# Посмотрим на размерность предсказаний
print(y_pred_1.shape, y_true.shape)

# Попробуем разные вектора предказаний и сравним их
print(f'Prediction 1: {loss_op(y_pred_1, y_true)}')
print(f'Prediction 2: {loss_op(y_pred_2, y_true)}')
print(f'Prediction 3: {loss_op(y_pred_3, y_true)}')

Отлично, с категориальной перекрестной энтропией мы уже встречались, по формуле на сайте можно увидеть, что реализация PyTorch включает в себя Softmax, так что на вход предсказания ожидаются сырые степени уверенности.

Сейчас мы убедились, что функция потерь работает корректно и мы освоили способ, как с ней работать. Настало время написать цикл обучения!

# Пора учить!

Как известно, лучшие практики предполагают разделение данных на выборки, чтобы можно было на обучающей проводить обучение, на валидационной искать наилучшие параметры и архитектуру, а на тестовой производить конечную оценку!

Давайте разделим наш `train_df` на выборки обучения, валидации и теста. Тест (test) должен быть 20% от всего набора, тогда останется 80% на обучение (train_val). Затем валидация (val) отделяется от полученной выборки обучения (train_val) на 20%, а оставшееся (train) используется для обучения. И не забывайте про стратификацию!

In [ ]:
from sklearn.model_selection import train_test_split

_train_val_df, _test_df = train_test_split(
    train_df, test_size=0.2, random_state=RANDOM_STATE,
    stratify=train_df['label'] 
)

_train_df, _val_df = train_test_split(
    _train_val_df, test_size=0.2, random_state=RANDOM_STATE,
    stratify=_train_val_df['label'] 
)

print(f'Train size: {_train_df.shape[0]}')
print(f'Val size: {_val_df.shape[0]}')
print(f'Test size: {_test_df.shape[0]}')

Замечательно! Теперь мы подготовили данные и можем подготовить загрузчики данных, чтобы затем использовать их в обучении!

In [ ]:
def prepare_loaders(batch_size):
    # Готовим загрузчики для каждой выборки
    train_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(_train_df),
        batch_size=batch_size,
        shuffle=True
    )

    val_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(_val_df),
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=DigitsDataset(_test_df),
        batch_size=batch_size,
        shuffle=True
    )

    return train_loader, val_loader, test_loader

Загрузчики готовы, теперь переходим к разбору основных циклов обучения и валидации! Для начала напишем функцию обучения одной эпохи:

In [ ]:
def train_epoch(model, loader, loss, optim):
    history = {
        'loss': []
    }

    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    model.train()
    for imgs_batch, labels_batch in loader:
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history['loss'].append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    return history

In [ ]:
# TEST

# Зададим еще пару констант
LR = 0.01
EPOCHS = 5
BATCH_SIZE=32

train_loader, val_loader, test_loader = prepare_loaders(BATCH_SIZE)

model = MyFirstConvNet()
loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=LR)

epoch_train_history = train_epoch(model, train_loader, loss_op, optim)

assert len(epoch_train_history['loss']) == len(train_loader)

Теперь перейдем к этапу валидации:

In [ ]:
def valid_epoch(model, loader, loss):
    history = {
        'loss': []
    }

    # Таким образом переводим модель в режим исполнения (inference)
    # В этом режиме отключены градиенты, он быстрее, 
    #   но в нем нельзя обучать модель
    model.eval()
    for imgs_batch, labels_batch in loader:
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history['loss'].append(loss.item())
    
    return history

In [ ]:
# TEST
epoch_val_history = valid_epoch(model, val_loader, loss_op)

assert len(epoch_val_history['loss']) == len(val_loader)

Теперь мы можем объединить написанный код в полный цикл обучения:

In [ ]:
# Зададим еще пару констант
LR = 0.01
EPOCHS = 2  # Проверочные 2 эпохи
BATCH_SIZE=32

train_loader, val_loader, test_loader = prepare_loaders(BATCH_SIZE)

model = MyFirstConvNet()
loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=LR)

history = {
    'train_loss': [],
    'val_loss': []
}

for epoch in range(EPOCHS):
    # Сначала выполняем цикл обучения
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim)
    # Затем исполняем цикл валидации
    epoch_val_history = valid_epoch(model, val_loader, loss_op)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history['loss'])
    epoch_val_loss = np.mean(epoch_val_history['loss'])
    history['train_loss'].append(epoch_train_loss)
    history['val_loss'].append(epoch_val_loss)

    print(f'Epoch {epoch}:')
    print(f'  Train loss: {epoch_train_loss}')
    print(f'  Valid loss: {epoch_val_loss}')

print(history)

Заметим два важных факта:
- Код работает и не выдает ошибок;
- Функция потерь уменьшается - это хороший знак!

На этот момент у нас написан код для загрузки данных, обучения, валидации и код модели! Вроде можно учить модель и далее оценивать ее работу, но мы еще не использовали всей мощи обучения. Пора посмотреть, как нашу модель учить с использованием GPU!


# Применение аппаратного ускорения для обучения и исполнения

PyTorch отличается очень удобным интерфейсом по реализации вычислений на конкретных ускорителях. Для примера, если вы работаете в Google Colab, то с этого момента рекомендуется проверить, что у вас включен GPU ускоритель: Среда выполнения -> Сменить среду выполнения -> Выберите аппаратный ускоритель GPU.

> Если вы работаете на своей машине и у вас нет GPU, то не беда. Мы будем писать код, который подстраивается под использование на том или ином вычислителе.

> Если вам не нужно в ноутбуке использовать GPU, то рекомендуется отключать ускоритель, чтобы не занимать важные ресурсы. Так код переходит на исполнение на CPU.

При смене ускорителя ваша среда перезапускается, поэтому удобно выполнить ячейки до этой в вкладке "Среда выполнения" или комбинацией Ctrl+F8.

> Помните, что при перезапуске среды ваш файл `kaggle.json` удалится, так что перед перезапуском надо загрузить его снова.

Для проверки доступности GPU PyTorch имеет удобный интерфейс, проверим его:

In [ ]:
# Данная булева проверка вернет True, 
#   если имеется CUDA устройство (GPU от NVidia)
torch.cuda.is_available()

Отлично, если проверка пройдена, то вы можете использовать CUDA устройство. Обычно для применения делается следующий выбор устройства из доступных:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Таким образом, мы получили объект устройства, на котором планируется исполнение. Теперь, чтобы выполнять вычисления на GPU (в том числе обучение), нужно, чтобы на вход модулей поступали тензоры, которые размещены на устройстве для вычисления. Для этого есть метод `torch.Tensor.to()`, в который можно передать объект устройства, на котором будем производить вычисления. Давайте создадим тензор и перенесем его на устройство:

In [ ]:
smpl_tensor = torch.tensor([1, 2, 3]).float()
print(smpl_tensor)

# При переносе тензора на устройство важно сделать присвоение,
#   так как тензор не сам переносится, а метод `.to()` возвращает
#   в качестве результата тензон на устройстве
smpl_tensor = smpl_tensor.to(device)
print(smpl_tensor)

По сути, для обучения на GPU требуется:
- Перенести модель на устройство (у `nn.Module` тоже есть метод `.to()`);
- Перенести входной тензор батча на устройство;
- Перенести тензор разметки на устройство;
- Произвести вычисление предсказания модели;
- Вычислить Loss (заметьте, все еще на устройстве);
- (*) Если предсказания планируется использовать в numpy, то надо вернуть тензор на CPU методом `.cpu()`.

Давайте напишем новые функции обучения эпох с учетом устройства для выполнения!

In [ ]:
def train_epoch(model, loader, loss, optim, device):
    history = {
        'loss': []
    }

    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    model.train()
    for imgs_batch, labels_batch in loader:
        imgs_batch = imgs_batch.to(device)
        labels_batch = labels_batch.to(device)

        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history['loss'].append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()
    
    return history

def valid_epoch(model, loader, loss, device):
    history = {
        'loss': []
    }

    # Таким образом переводим модель в режим исполнения (inference)
    # В этом режиме отключены градиенты, он быстрее, 
    #   но в нем нельзя обучать модель
    model.eval()
    for imgs_batch, labels_batch in loader:
        imgs_batch = imgs_batch.to(device)
        labels_batch = labels_batch.to(device)
        
        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        history['loss'].append(loss.item())
    
    return history

In [ ]:
# TEST
LR = 0.01
EPOCHS = 2  # Проверочные 2 эпохи
BATCH_SIZE=32

train_loader, val_loader, test_loader = prepare_loaders(BATCH_SIZE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyFirstConvNet()
model.to(device)

loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=LR)

history = {
    'train_loss': [],
    'val_loss': []
}

for epoch in range(EPOCHS):
    # Сначала выполняем цикл обучения
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim, device)
    # Затем исполняем цикл валидации
    epoch_val_history = valid_epoch(model, val_loader, loss_op, device)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history['loss'])
    epoch_val_loss = np.mean(epoch_val_history['loss'])
    history['train_loss'].append(epoch_train_loss)
    history['val_loss'].append(epoch_val_loss)

    print(f'Epoch {epoch}:')
    print(f'  Train loss: {epoch_train_loss}')
    print(f'  Valid loss: {epoch_val_loss}')

print(history)

Теперь после добавления выбора устройства (точнее выбор GPU, если доступен, иначе CPU) мы можем ускорить наше обучение с помощью GPU или как минимум работать на всегда доступном CPU!

# А как оценивать то?

Во время обучения значение функции потерь на валидации - дело, конечно, полезное, но не совсем понятное. Эти значения нельзя никак интерпретировать. Поэтому, дополнительно применяется оценка по определенным метрикам, которые позволят понять хотя бы банально accuracy нашей модели на конкретной эпохе. Давайте по аналогии с функциями `train_epoch()` или `valid_epoch()` напишем функцию `evaluate_loader()`, который будет возвращать словарь с желаемыми метриками. Для начала это будет accuracy.

Мы сначала напишем функцию оценки accuracy по тензорам предсказания и разметки:

In [ ]:
# TODO
def evaluate_batch_accuracy(y_pred, y_true):
    '''
    y_pred - батч сырых степеней уверенности, размер (N, K)
    y_true - вектор истинных значений, размер (N)
    '''
    return None

In [ ]:
# TEST

y_true = torch.tensor([1, 2, 1])
y_pred = torch.tensor([
    [0, 0, 0, 2],
    [10, 10, 12, 0],
    [1, 10, -1, 2]
]).float()

assert np.isclose(evaluate_batch_accuracy(y_pred, y_true), 0.666666)

А теперь можно написать функцию оценки загрузчика:

In [ ]:
# TODO
def evaluate_loader(model, loader, device):
    metrics = {
        'accuracy': []
    }

    model.eval()
    for imgs_batch, labels_batch in loader:
        labels_batch = labels_batch.to(device)
        
    return metrics

In [ ]:
# TEST
BATCH_SIZE=32

_, val_loader, _ = prepare_loaders(BATCH_SIZE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyFirstConvNet()
model.to(device)
epoch_metrics = evaluate_loader(model, val_loader, device)

epoch_mean_accuracy = np.mean(epoch_metrics['accuracy'])

assert np.isclose(epoch_mean_accuracy, 0.0903273)

Необученная модель должна давать показатель порядка 10% (0.1), что соответствует случайным предсказаниям 1 из 10 классов.

Превосходно! Мы теперь можем еще и оценивать метрику, которая более понятно описывает работу модели на каждой эпохе! Осталось теперь только запустить обучение и посмотреть на результаты!

# Пора учить по полной

Давайте выстроим весь наш цикл и постараемся обучить модель с учетом оценки Loss на выборке для обучения и Loss + accuracy на выборке валидации. Тестовая выборка остается нетронутой!

In [ ]:
LR = 0.01
EPOCHS = 50
BATCH_SIZE=128  # Возьмем побольше

train_loader, val_loader, test_loader = prepare_loaders(BATCH_SIZE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyFirstConvNet()
model.to(device)

loss_op = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=LR)

history = {
    'train_loss': [],
    'val_loss': [],
    'accuracy': []
}

for epoch in range(EPOCHS):
    epoch_train_history = train_epoch(model, train_loader, loss_op, optim, device)
    epoch_val_history = valid_epoch(model, val_loader, loss_op, device)
    epoch_metrics = evaluate_loader(model, val_loader, device)

    # В историю обучения попадает среднее по эпохе значение
    epoch_train_loss = np.mean(epoch_train_history['loss'])
    epoch_val_loss = np.mean(epoch_val_history['loss'])
    epoch_accuracy = np.mean(epoch_metrics['accuracy'])

    history['train_loss'].append(epoch_train_loss)
    history['val_loss'].append(epoch_val_loss)
    history['accuracy'].append(epoch_accuracy)

    print(f'Epoch {epoch}:')
    print(f'  Train loss: {epoch_train_loss}')
    print(f'  Valid loss: {epoch_val_loss}')
    print(f'  Accuracy: {epoch_accuracy}')

In [ ]:
plt.figure(figsize=[20,10])

plt.subplot(121)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Validation')
plt.title('Loss')
plt.grid()

plt.subplot(122)
plt.plot(history['accuracy'])
plt.title('Accuracy')
plt.grid()

plt.show()

Процесс не быстрый, но всё-таки это уже глубокое обучение! Некоторые сети могут учиться неделями или месяцами без перерыва!

Смотря на графики, как вы считаете, хватило ли 50 эпох, чтобы полностью обучиться модели? Попробуйте поменять параметры (размер батча, коэффициент обучения, количество эпох) и получить лучшие показатели модели!

> Подумайте, как в данной ситуации понять, что случилось переобучение/недообучение (overfit/underfit)?

# Оценка модели на тесте, слоеном тесте

После того, как найдена наилучшая модель (вот для чего нужна выборка валидации), следующим этапом является оценка модели на тестовой выборке. Так мы поймем, не слишком ли модель заточена под те данные, которые были использованы при обучении и валидации.
> Этот шаг также может показать, что тест координально отличается от остальных выборок, хотя он должен быть самым аккуратно подготовленным, так как на нем производится окончательная оценка!


In [ ]:
# Здесь просто подкладываем модель (лучшую) и загрузчик данных теста
epoch_metrics = evaluate_loader(model, test_loader, device)

epoch_mean_accuracy = np.mean(epoch_metrics['accuracy'])
print(f'Test acuracy: {epoch_mean_accuracy}')

Значения показателей на выборке для тестирования могут отличаться от показателей на выборке валидации, но главное, чтобы не было слишком больших отличий!

В этот момент мы можем закончить работу с моделью и сказать, что модель обучена хорошо (если у вас показатель на тесте выше 90%), но остался очень важный шаг - анализ ошибок!

# Анализ ошибок (вместе с визуальной оценкой)

Этот этап является не менее важным, чем обучение, так как модель никогда не будет давать 100% результат на больших данных (иначе слишком подозрительно и надо все еще раз все проверить). Поэтому мы должны понять, в каких ситуация ошибается модель и можем ли мы исправить данные/улучшить модель?

Перво-наперво, нужно построить матрицу ошибок, то есть получить полный вектор предсказаний и полный вектор разметки для построения. Пишем!

In [ ]:
y_true = []
y_pred = []
err_img_triplets = []

model.eval()
for imgs_batch, labels_batch in test_loader:
    imgs_batch = imgs_batch.to(device)
    labels_batch = labels_batch.to(device)
    
    pred = model(imgs_batch)
    _, pred_labels = torch.max(pred, dim=1)
    
    for i in range(imgs_batch.shape[0]):
        y_true.append(labels_batch[i].item())
        y_pred.append(pred_labels[i].item())

        if y_true[-1] != y_pred[-1]:
            err_img_triplets.append(
                (imgs_batch[i].cpu(), y_true[-1], y_pred[-1])
            )

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mtrx = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_mtrx, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Ground Truth')
plt.show()

Видим, что в большинстве случаев предсказания корректны, но есть промахи, когда цифра 4 принята как 9, цифра 7 за 9, а также путается 3 и 8. Согласитесь, в некоторых случаях ошибки логичны, но важно взглянуть на них визуально!

Теперь построим отчет классификации:

In [ ]:
from sklearn.metrics import classification_report

rep = classification_report(y_true, y_pred, digits=4)
print(rep)

И наконец взглянем на наши ошибки!

In [ ]:
plt.figure(figsize=[25, 10])
subplot = 1

for img_tnsr, true_label, pred_label in err_img_triplets[:5]:
    img = img_tnsr.permute(1, 2, 0).mul(255).numpy().astype(np.uint8)

    plt.subplot(1, 5, subplot)
    subplot += 1

    plt.imshow(img[..., 0], cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(f'True: {true_label}, predicted: {pred_label}')

plt.show()

Видите, в одном случае вообще цифра похожа на 9, что и предсказывает модель, но разметка говорит, что это 3! Здесь либо разметка содержит ошибку, либо так и выглядит.

Визуальный анализ полезен тем, что позволяет понять, на каких случаях ошибается модель и насколько случаи выпадают из стандартных кейсов.

# Что мне делать со своими картинками?

Если вы хотите работать со своими изображениями, то необходимо написать класс, который будет выполнять предсказание по конкретной картинке. В данном случае сделать это несложно, поэтому разбираемся! Для начала загрузим из тестовой выборки с Kaggle (в нашем случае она не имеет разметки) набор данных:

In [ ]:
test_df = pd.read_csv('dataset/test.csv')

imgs_batch_flattened = test_df.to_numpy()
imgs_batch = imgs_batch_flattened.reshape(-1, 28, 28, 1).astype(np.uint8)

Теперь напишем класс, который будет выполнять пресказание, на вход нужно лишь подать картинку:

In [ ]:
# TODO - реализуйте класс для предсказания цифры по картинке
class ImageInference:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        # Переведите модель на устройство

    def infer_image(self, image):
        # Переведите модель в режим inference (.eval())

        # Преобразовать изображение в тензор с подготовкой в формате CHW
        # Преобразовать в батч: CHW -> NCHW

        # Не забывайте, что применение данных вне фреймворка PyTorch 
        #   требует возврата на CPU методом .cpu()

        return pred_label

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inferencer = ImageInference(model, device)

IMAGE_IDX = 10
sample_img = imgs_batch[IMAGE_IDX]

pred_label = inferencer.infer_image(sample_img)

plt.imshow(sample_img[..., 0], cmap='gray')
plt.title(f'Predicted: {pred_label}')
plt.xticks([])
plt.yticks([])
plt.show()

Если вы видите цифру 5 - великолепно! Значит все загружено правильно, цифра отобразилась и над ней должно появиться предсказание модели!

# Вопросы (заметки)

- Как сделать так, чтобы можно было задать количество предсказываемых классов модели? Например, сейчас, если поменяется количество предсказываемых классов - нужно менять код модели.
- Как сделать так, чтобы модель учитывала размер входа при формировании полносвязного слоя? Например, сейчас, если поменяется размер входа на 32х32 - нужно менять код модели.

